# Estudo de Caso: Alocação de Ativos em Carteira de Investimentos

Monte uma carteira de ações para o mês de Setembro/2020 baseado na Moderna Teoria de Portfolio (Markowitz, 1952), a qual estuda a melhor combinação possível dos ativos analisados e sugere uma alocação de ativos dentro de uma carteira que maximize o retorno. 

Considere as cotações históricas das ações que fazem parte da [Carteira Teórica do Ibovespa em 01/09/2020](http://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-ibovespa-ibovespa-composicao-da-carteira.htm) disponíveis em [ibovespa_desc.csv](ibovespa_desc.csv) e no diretório ibovespa.

Caso queira consultar o retorno histórico diário, semanal ou mensal acesse o [link](https://www.investing.com/equities/ambev-pn-historical-data).

In [1]:
#include <iostream>
#include <iomanip>

#include "csv.h"
#include "setup.h"

#include "ortools/linear_solver/linear_solver.h"
#include "ortools/linear_solver/linear_solver.pb.h"

using namespace operations_research;

## Símbolos de uma carteira

Use a função getTickersByStartDate para retornar a lista de símbolos dos ativos existentes a partir de um determinado mês e ano.

In [2]:
int getTickersByStartDate(int year, int month, std::vector<std::string> & selectedTickers)
{
    int months(0);
    std::string initial_date(std::to_string(year) + "-" + (month<10 ? "0" + std::to_string(month) : std::to_string(month)) );
    std::ifstream f1("ibovespa_desc.csv");
    auto tickers = csv_read_string(f1);
    transpose<std::string>(tickers);
    f1.close();

    for(int ticker = 0; ticker < tickers[0].size(); ticker++){
        auto filename = std::string("ibovespa/") + tickers[0][ticker] + "M.csv";
        std::ifstream f2(filename);
        auto returns = csv_read_string(f2);
        transpose<std::string>(returns);
        f2.close();
        
        int date_offset = -1;
        for(int month=0; month<returns[0].size(); month++){
            if (returns[0][month] == initial_date){
                date_offset = month;
                break;
            } 
            else if (returns[0][month] > initial_date){
                date_offset = -1;
                break;
            }
        }
        
        if (date_offset >= 0) {
            months = returns[0].size()-date_offset;
            selectedTickers.push_back(tickers[0][ticker]);
        }

    }
    return months;
}

In [3]:
std::vector<std::string> tickers;
int months = getTickersByStartDate(2020, 1, tickers);

In [4]:
//PROFESSOR
//std::vector<std::string> tickers {"ABEV3","BRAP4","CSNA3","GGBR4","NTCO3","QUAL3","RADL3","RENT3","VALE3","WEGE3"};
//int months = 9;

In [5]:
tickers

{ "ABEV3", "AZUL4", "B3SA3", "BBAS3", "BBDC3", "BBDC4", "BBSE3", "BEEF3", "BPAC11", "BRAP4", "BRDT3", "BRFS3", "BRKM5", "BRML3", "BTOW3", "CCRO3", "CIEL3", "CMIG4", "COGN3", "CPFE3", "CRFB3", "CSAN3", "CSNA3", "CVCB3", "CYRE3", "ECOR3", "EGIE3", "ELET3", "ELET6", "EMBR3", "ENBR3", "ENGI11", "EQTL3", "FLRY3", "GGBR4", "GNDI3", "GOAU4", "GOLL4", "HAPV3", "HGTX3", "HYPE3", "IGTA3", "IRBR3", "ITSA4", "ITUB4", "JBSS3", "KLBN11", "LAME4", "LREN3", "MGLU3", "MRFG3", "MRVE3", "MULT3", "NTCO3", "PETR3", "PETR4", "QUAL3", "RADL3", "RAIL3", "RENT3", "SANB11", "SBSP3", "SULA11", "TAEE11", "TIMP3", "TOTS3", "UGPA3", "USIM5", "VALE3", "VIVT4", "VVAR3", "WEGE3", "YDUQ3" }

## Retorno de uma carteira

Calcule o retorno médio mensal em percentual de cada ação baseado nos dados históricos fornecidos. Fica a critério de cada grupo escolher o intervalo mais apropriado de datas.

    O algoritmo abaixo seleciona somente as ações de empresas listadas na Bovespa a partir de uma data configurada (initial_date).

In [6]:
void getReturnsByTicker(const std::vector<std::string> & tickers, std::vector<std::vector<double>> & selectedReturns)
{
    for(int ticker = 0; ticker < tickers.size(); ticker++){
        auto filename = std::string("ibovespa/") + tickers[ticker] + "M.csv";
        std::ifstream f2(filename);

        std::cout << "Reading: " << filename << std::endl;

        auto returns = csv_read_string(f2);
        transpose<std::string>(returns);
        f2.close();
        
        int date_offset = returns[1].size() - selectedReturns[ticker].size();
        
        if (date_offset >= 0) {
            for(int month=date_offset; month<returns[1].size(); month++) {
                selectedReturns[ticker][month-date_offset] = std::stof(returns[1][month]);
            }
        }

    }

}

In [7]:
std::vector<std::vector<double>> returns(tickers.size(), std::vector<double>(months));
getReturnsByTicker(tickers, returns);

Reading: ibovespa/ABEV3M.csv
Reading: ibovespa/AZUL4M.csv
Reading: ibovespa/B3SA3M.csv
Reading: ibovespa/BBAS3M.csv
Reading: ibovespa/BBDC3M.csv
Reading: ibovespa/BBDC4M.csv
Reading: ibovespa/BBSE3M.csv
Reading: ibovespa/BEEF3M.csv
Reading: ibovespa/BPAC11M.csv
Reading: ibovespa/BRAP4M.csv
Reading: ibovespa/BRDT3M.csv
Reading: ibovespa/BRFS3M.csv
Reading: ibovespa/BRKM5M.csv
Reading: ibovespa/BRML3M.csv
Reading: ibovespa/BTOW3M.csv
Reading: ibovespa/CCRO3M.csv
Reading: ibovespa/CIEL3M.csv
Reading: ibovespa/CMIG4M.csv
Reading: ibovespa/COGN3M.csv
Reading: ibovespa/CPFE3M.csv
Reading: ibovespa/CRFB3M.csv
Reading: ibovespa/CSAN3M.csv
Reading: ibovespa/CSNA3M.csv
Reading: ibovespa/CVCB3M.csv
Reading: ibovespa/CYRE3M.csv
Reading: ibovespa/ECOR3M.csv
Reading: ibovespa/EGIE3M.csv
Reading: ibovespa/ELET3M.csv
Reading: ibovespa/ELET6M.csv
Reading: ibovespa/EMBR3M.csv
Reading: ibovespa/ENBR3M.csv
Reading: ibovespa/ENGI11M.csv
Reading: ibovespa/EQTL3M.csv
Reading: ibovespa/FLRY3M.csv
Reading: ibo

# Retorno Médio e Risco de uma carteira

Calcule a matriz de covariância considerando os retornos selecionados no passo anterior. 

Suponha que sejam consideradas $n$ ações para inclusão nessa carteira e façamos com que as variáveis de decisão $x_i$ ($i = 1, 2, ..., n$) representem o percentual da carteira que será alocado no ativo $i$. 

Estipulamos que $\mu_i$ e $\sigma_{ii}$ sejam, respectivamente, a média e a variância, (estimadas) do retorno sobre cada cota da ação i, em que $\sigma_{ii}$ mede o risco dessa ação.

Para $i=1,2,...,n$ ($i \ne j$),façamos com que $\sigma_{ij}$ represente a co-variância do retorno sobre cada cota da ação i e j. 

Como seria difícil estimar todos os valores $\sigma_{ij}$, a metodologia usual é partir de certas hipóteses sobre o comportamento do mercado que nos permitam calcular $\sigma_{ij}$ diretamente de $\sigma_{ii}$ e $\sigma_{jj}$.

A seguir, o valor esperado $R(x)$ e a variância $V(x)$ do retorno total de toda a carteira são:

$R(x) = \sum_{j=1}^n \mu_{j}x_j$

e

$V(x) = \sum_{i=1}^n \sum_{j=1}^n \sigma_{ij}x_i x_j$

em que $V(x)$ mede o risco associado à carteira com base na matriz de covariância $\Sigma$, $N\times N$:

$$ \Sigma = \left[\begin{matrix}
VAR(R_1) & COV(R_1, R_2) & \cdots & COV(R_1, R_N) \\
COV(R_2, R_0) & VAR(R_2) & \cdots & COV(R_2, R_N) \\
\vdots & \vdots & \ddots & \vdots \\
COV(R_N, R_1) & COV(R_N, R_2) & \cdots & VAR(R_N)
\end{matrix}\right] $$


In [8]:
void getMeanCovariance(const std::vector<std::vector<double>> &x, std::vector<double> &average, std::vector<std::vector<double>> &covmatrix){
    auto const n = x.size(); //tickers
    auto const m = x[0].size()-1; //months
    for (int i = 0; i < n; i++){
        double sum = 0;
        for (int k = 0; k < m; k++)
            sum += x[i][k];
        average.push_back(sum / (double)m);
    }

    for (int i = 0; i < n; i++){
        std::vector<double>temp;
        for (int j = 0; j < n; j++){
            double sum = 0;
            for (int k = 0; k < m; k++)
                sum += (x[i][k] - average[i])*(x[j][k] - average[j]);
            temp.push_back(sum / (double)(m));
        }
        covmatrix.push_back(temp);
    }
}

In [9]:
auto getPortfolioReturn(const std::vector<std::string> &portfolio_tickers, 
                        const std::vector<double> &portfolio_weights){
    auto portfolio_returns = 0.0;
    for(int i=0; i < portfolio_tickers.size(); i++){
        for(int j=0; j < tickers.size(); j++) {
            if (portfolio_tickers[i] == tickers[j]) {
                portfolio_returns += portfolio_weights[i] * returns[j][returns[j].size()-1];
            }
        }
    }
    return portfolio_returns;
}

## Otimização de Carteira de Investimentos

O objetivo é analisar as cotações históricas das ações do Ibovespa segundo a Moderna Teoria de Portfolio para gerar uma distribuição de alocação ótima (em percentual) composta de 5 a 10 ativos para ser utilizada na carteira mensal de Setembro de 2020. 

**Importante:** Os grupos cujas carteiras otimizadas alcançarem os maiores retornos (positivos) em Setembro/2020 receberão pontuação extra proporcional ao rendimento alcançado.


In [10]:
std::vector<double> mean;
std::vector<std::vector<double>> covariance;
getMeanCovariance(returns, mean, covariance);

In [11]:
std::vector<std::string> portfolio_tickers;
std::vector<double> portfolio_weights;

## Escolha uma das duas modelagens abaixo e justifique:

Minimizar: $x^T\sum x$

Minimizar: $\sum_{i=1}^n \sum_{j=1}^n \sigma_{ij}x_i x_j$


Sujeito a: 

$
\begin{align}
\mu^Tx \ge r_{min} \\
1^Tx \le 1  \\
x \ge 0
\end{align}
$

In [12]:
{
    const double infinity = MPSolver::infinity();
    
    // max and min portfolio allocation
    auto net_upper_long_bound=1.0;
    auto net_lower_long_bound=0.5;
    auto r_min = 0.05;
    auto n = tickers.size();
    
    MPModelRequest model_request;
    
    model_request.set_solver_type(MPModelRequest::SCIP_MIXED_INTEGER_PROGRAMMING);
    
    auto model_proto = model_request.mutable_model();

    model_proto->set_name("Min Portfolio Otimization");
    model_proto->set_maximize(false);
   
    std::vector<MPVariableProto*> x(n);
    for(auto i = 0; i < n; i++) {
        x[i] = model_proto->add_variable();
        x[i]->set_name(tickers[i]);  
        x[i]->set_upper_bound(1);  
        x[i]->set_lower_bound(0.05);//long only
        x[i]->set_objective_coefficient(0); //FO: 0
        x[i]->set_is_integer(false);      
    }
    
    //FO: sum(sigma[i][j] * x[i] * x[j])
    auto quad_obj = model_proto->mutable_quadratic_objective();
    for(auto i = 0; i < n; i++) {
        for(auto j = 0; j < n; j++) {
            quad_obj->add_qvar1_index(i);
            quad_obj->add_qvar2_index(j);
            quad_obj->add_coefficient(covariance[i][j]);
        }
    }
    
    //Constraint: net_lower_long_bound <= sum(x[i]) <= net_upper_long_bound
    auto c = model_proto->add_constraint();
    c->set_name("c");  
    c->set_lower_bound(net_lower_long_bound);    
    c->set_upper_bound(net_upper_long_bound);
    for(auto i = 0; i < n; i++) {
        c->add_var_index(i);
        c->add_coefficient(1);
    }
    
    //Constraint: sum(mean_return[i] * x[i]) >= r_min
    auto r = model_proto->add_constraint();
    r->set_name("r");  
    r->set_lower_bound(r_min);    
    r->set_upper_bound(infinity);
    for(auto i = 0; i < n; i++) {
        r->add_var_index(i);
        r->add_coefficient(mean[i]);
    }
    
    MPSolutionResponse solution_response;
    MPSolver::SolveWithProto(model_request, &solution_response);

    if (solution_response.status() == MPSOLVER_OPTIMAL) {
        std::cout << "Solução ótima encontrada!" << std::endl;
    
        std::vector<double> solution(n);
        for (int i = 0; i < n; i++) 
            solution[i] = solution_response.variable_value(i);

        double rs = std::inner_product(mean.begin(), mean.end(), solution.begin(), 0.0);

        std::cout << "Retorno = " << std::fixed << std::setprecision(2) << rs*100.0 << "% (a.m.) " << std::endl;

        portfolio_tickers.clear();
        portfolio_weights.clear();
        for (int i = 0; i < n; i++) {
            if (solution[i] >= 0.01) {
                portfolio_tickers.push_back(tickers[i]);
                portfolio_weights.push_back(solution[i]);
                std::cout << tickers[i] << " = " << solution[i] << std::endl;
            }
        }
    } else {
        std::cout << "Solução não encontrada!" << std::endl;
    }
    
}

Solução não encontrada!


In [13]:
std::cout << getPortfolioReturn(portfolio_tickers, portfolio_weights)*100.0 << "% (a.m.)";

0% (a.m.)


Maximizar: $\mu^Tx - \rho x^T\sum x$

Maximizar: $\sum_{i=1}^n \mu_i x_i - \rho \sum_{i=1}^n \sum_{j=1}^n \sigma_{ij}x_i x_j$


Sujeito a: 

$
\begin{align}
1^Tx \le 1  \\
x \ge 0
\end{align}
$

In [14]:
{
    // max and min portfolio allocation
    auto net_upper_long_bound=1.0;
    auto net_lower_long_bound=0.5;
    
    // risk/reward tradeoff input, higher means less risk, b/t .1 & 1
    auto rho = 1.0;
    
    //std::cout << "Informe um dos valores para rho [0.1, 0.5, 1.0, 10.0]:";
    //std::cin >> rho;
        
    auto n = tickers.size();
    
    MPModelRequest model_request;
    
    model_request.set_solver_type(MPModelRequest::SCIP_MIXED_INTEGER_PROGRAMMING);
    
    auto model_proto = model_request.mutable_model();

    model_proto->set_name("Portfolio Otimization");
    model_proto->set_maximize(true);
   
    std::vector<MPVariableProto*> x(n);
    for(auto i = 0; i < n; i++) {
        x[i] = model_proto->add_variable();
        x[i]->set_name(tickers[i]);  
        x[i]->set_upper_bound(1);  
        x[i]->set_lower_bound(0.05);//long only
        x[i]->set_objective_coefficient(mean[i]); //FO: sum(mean_return[i] * x[i])
        x[i]->set_is_integer(false);      
    }
    
    //FO: - rho * sum(sigma[i][j] * x[i] * x[j])
    auto quad_obj = model_proto->mutable_quadratic_objective();
    for(auto i = 0; i < n; i++) {
        for(auto j = 0; j < n; j++) {
            quad_obj->add_qvar1_index(i);
            quad_obj->add_qvar2_index(j);
            quad_obj->add_coefficient(-rho*covariance[i][j]);
        }
    }
    
    //Constraint: net_lower_long_bound <= sum(x[i]) <= net_upper_long_bound
    auto c = model_proto->add_constraint();
    c->set_name("c");  
    c->set_lower_bound(net_lower_long_bound);    
    c->set_upper_bound(net_upper_long_bound);
    for(auto i = 0; i < n; i++) {
        c->add_var_index(i);
        c->add_coefficient(1);
    }
    
    MPSolutionResponse solution_response;
    MPSolver::SolveWithProto(model_request, &solution_response);

    if (solution_response.status() == MPSOLVER_OPTIMAL) {
        std::cout << "Solução Ótima!" << std::endl;
    
        std::vector<double> solution(2*n);
        for (int i = 0; i < n; i++) 
            solution[i] = solution_response.variable_value(i);

        double rs = std::inner_product(mean.begin(), mean.end(), solution.begin(), 0.0);

        std::cout << "Retorno = " << std::fixed << std::setprecision(2) << rs*100.0 << "% (a.m.) " << std::endl;

        portfolio_tickers.clear();
        portfolio_weights.clear();
        for (int i = 0; i < n; i++) {
            if (solution[i] >= 0.01) {
                portfolio_tickers.push_back(tickers[i]);
                portfolio_weights.push_back(solution[i]);
                std::cout << tickers[i] << " = " << solution[i] << std::endl;
            }
        }
    } else {
        std::cout << "Solução não encontrada!" << std::endl;
    }
    
}

Solução não encontrada!


## A otimização deverá retornar o vetor $x$ contendo apenas entre 5 e 10 ativos selecionados automaticamente pelo solver.

In [15]:
std::cout << getPortfolioReturn(portfolio_tickers, portfolio_weights)*100.0 << "% (a.m.)";

0% (a.m.)

In [20]:
//GRUPO 1
{
    std::vector<std::string> portfolio_tickers {"BTOW3","EQTL3","MGLU3","RADL3","VVAR3","WEGE3"};
    std::vector<double> portfolio_weights {0.15,0.17,0.17,0.17,0.15,0.17};
    std::cout << getPortfolioReturn(portfolio_tickers, portfolio_weights_G1)*100.0 << "% (a.m.)";
}

-5.69298% (a.m.)

In [19]:
//GRUPO 2
{
    std::vector<std::string> portfolio_tickers {"BTOW3","CSNA3","ELET3","ELET6","MGLU3","VVAR3"};
    std::vector<double> portfolio_weights {0.05,0.20,0.20,0.15,0.20,0.20};
    std::cout << getPortfolioReturn(portfolio_tickers, portfolio_weights)*100.0 << "% (a.m.)";
}

-8.11311% (a.m.)

In [22]:
//GRUPO 3
{
    std::vector<std::string> portfolio_tickers {"BTOW3","EQTL3","MGLU3","VVAR3","WEGE3"};
    std::vector<double> portfolio_weights {0.20,0.20,0.20,0.20,0.20};
    std::cout << getPortfolioReturn(portfolio_tickers, portfolio_weights)*100.0 << "% (a.m.)";
}

-9.27689% (a.m.)

In [23]:
//GRUPO 4
{
    std::vector<std::string> portfolio_tickers {"BTOW3","MGLU3","MRFG3","VVAR3","WEGE3"};
    std::vector<double> portfolio_weights {0.20,0.20,0.20,0.20,0.20};
    std::cout << getPortfolioReturn(portfolio_tickers, portfolio_weights)*100.0 << "% (a.m.)";
}

-10.4007% (a.m.)

In [24]:
//GRUPO 5
{
    std::vector<std::string> portfolio_tickers {"EQTL3","HGTX3","LREN3","RADL3","VVAR3","WEGE3"};
    std::vector<double> portfolio_weights {0.36,0.04,0.14,0.09,0.08,0.30};
    std::cout << getPortfolioReturn(portfolio_tickers, portfolio_weights)*100.0 << "% (a.m.)";
}

-4.6029% (a.m.)

In [25]:
//GRUPO 6
{
    std::vector<std::string> portfolio_tickers {"EQTL3","HGTX3","LREN3","RADL3","VVAR3","WEGE3"};
    std::vector<double> portfolio_weights {0.20,0.082,0.20,0.1117,0.20,0.20};
    std::cout << getPortfolioReturn(portfolio_tickers, portfolio_weights)*100.0 << "% (a.m.)";
}

-5.91062% (a.m.)